In [1]:
"""
Faces recognition example

The dataset used in this example is a preprocessed excerpt of the
"Labeled Faces in the Wild", aka LFW_:
http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)
.. _LFW: http://vis-www.cs.umass.edu/lfw/
original source: http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html
"""

print __doc__


Faces recognition example

The dataset used in this example is a preprocessed excerpt of the
"Labeled Faces in the Wild", aka LFW_:
http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)
.. _LFW: http://vis-www.cs.umass.edu/lfw/
original source: http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html



In [2]:
from time import time
import logging
import pylab as pl
import numpy as np


from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import RandomizedPCA
from sklearn.svm import SVC

#Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

C:\Users\Carlos\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Carlos\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
#download the data and load it as numpy arrays.
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

#introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape
np.random.seed(42)

# for machine learning we use the data directly 
# (as relative pixel position info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# The label to predict is the ide of the person.
y = lfw_people.target
target_names = lfw_people.target_names
n_Classes = target_names.shape[0]

print"Total dataset size"
print "n_samples: %d" % n_samples
print "n_features: %d" % n_features
print "n_Classes: %d" % n_Classes

2018-02-17 23:09:22,595 Loading LFW people faces from C:\Users\Carlos\scikit_learn_data\lfw_home


Total dataset size
n_samples: 1288
n_features: 1850
n_Classes: 7


In [4]:
# split into a training and testing set
X_train, X_Test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled 
# dataset) : unsupervised feature extraction / dimentionality reduction
n_components =  150

#print "extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
t0 = time()
pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X_train)
print "done in %0.3fs" % (time() - t0)

eigenfaces = pca.components_.reshape((n_components, h, w))

print "Projecting the input data on the eigenfaces orthonormal basis"
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_Test)

print "done in %0.3fs" %(time() - t0)

done in 0.137s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.024s


C:\Users\Carlos\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


In [6]:
#Train a SVM classification model

print "fitting the classifier to the trainning set"
t0 = time()
param_grid={
    'C':[1e3,5e3,1e4,5e4,1e5],
    'gamma':[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}
# for sklearn version 0.16 or prior, the class_weight parameter is 'auto'
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid) #to check if is going wrong
clf = clf.fit(X_train_pca, y_train)

print "done in %0.3fs" % (time() - t0)
print "Best estimator found by grid search: "
print clf.best_estimator_

fitting the classifier to the trainning set
done in 13.313s
Best estimator found by grid search: 
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [ ]:
#Quantitave evaluation of the model quality on the test set

print "predicting the people names on the testing set"
t0 = time()
y_pred = clf.predict(X_test_pca)
print "done in %0.3fs" % (time() - t0)

print classification_report(y_test, y_pred, target_names = target_names)
print confusion_matrix(y_test, y_pred, labels=range(n_Classes))

In [ ]:
# Qualitative evaluation  of the predictions using matplotlib

def plot_gallery(images, titles, h, w,  n_row=3, n_col=4):
    """
    helper function to plot a gallery of portraits
    """
    pl.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    pl.subplots_adjust(bottom=0, left=0.1, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        pl.subplot(n_row, n_col, i + 1)
        pl.imshow(images[i].reshape((h, w)), cmap = pl.cm.gray)
        pl.title(titles[i], size=12)
        pl.xticks(())
        pl.yticks(())

In [ ]:
# plot the result of( the prediction on a portion of the test set
def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)


prediction_titles = [title(y_pred, y_test, target_names, i)
                        for i in range(y_pred.shape[0])]

plot_gallery(X_Test, prediction_titles, h, w)

In [ ]:
# plot the gallery of the most significative eigenfaces
eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

pl.show()

In [ ]:
# get the Variance of Each PC
# how much of the variance is explained by the first principal component?
arrvar = pca.explained_variance_ratio_ # : array, shape (n_components,)
arrvar